<a href="https://www.kaggle.com/code/chandlertimm/manga-colorization-training?scriptVersionId=96813749" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import json
import cv2
import matplotlib
import matplotlib.pyplot as plt
import scipy
import warnings
import shutil
import random
import shutil 

from PIL import Image
from tqdm import tqdm
from distutils.dir_util import copy_tree

# Split the dataset

In [ ]:
if not os.path.isdir('./manga_dataset'):
    os.makedirs('./manga_dataset')
    os.makedirs('./manga_dataset/colored/')
    os.makedirs('./manga_dataset/grayscale/')

In [ ]:
cl_1 = '../input/unified/blackclover/colored/'
cl_2 = '../input/unified/bukonohero/colored/'
cl_3 = '../input/unified/onepiece/colored/'

target_dir_cl = './manga_dataset/colored/'
    
file_names_cl_1 = os.listdir(cl_1)
file_names_cl_2 = os.listdir(cl_2)
file_names_cl_3 = os.listdir(cl_3)
    
for file_name in tqdm(file_names_cl_1, 'blackclover colored'):
    shutil.copy(os.path.join(cl_1, file_name), target_dir_cl)
    os.rename(os.path.join(target_dir_cl, file_name), os.path.join(target_dir_cl, str('blackclover_cl') + file_name))
    
for file_name in tqdm(file_names_cl_2, 'bukonohero colored'):
    shutil.copy(os.path.join(cl_2, file_name), target_dir_cl)
    os.rename(os.path.join(target_dir_cl, file_name), os.path.join(target_dir_cl, str('bukonohero_cl') + file_name))
    
for file_name in tqdm(file_names_cl_3, 'onepiece colored'):
    shutil.copy(os.path.join(cl_3, file_name), target_dir_cl)
    os.rename(os.path.join(target_dir_cl, file_name), os.path.join(target_dir_cl, str('onepiece_cl') + file_name))

In [ ]:
bw_1 = '../input/unified/blackclover/grayscale/'
bw_2 = '../input/unified/bukonohero/grayscale/'
bw_3 = '../input/unified/onepiece/grayscale/'

target_dir_bw = './manga_dataset/grayscale/'

file_names_bw_1 = os.listdir(bw_1)
file_names_bw_2 = os.listdir(bw_2)
file_names_bw_3 = os.listdir(bw_3)

for file_name in tqdm(file_names_bw_1, 'blackclover grayscale'):
    shutil.copy(os.path.join(bw_1, file_name), target_dir_bw)
    os.rename(os.path.join(target_dir_bw, file_name), os.path.join(target_dir_bw, str('blackclover_bw') + file_name))
    
for file_name in tqdm(file_names_bw_2, 'bukonohero grayscale'):
    shutil.copy(os.path.join(bw_2, file_name), target_dir_bw)
    os.rename(os.path.join(target_dir_bw, file_name), os.path.join(target_dir_bw, str('bukonohero_bw') + file_name))
    
for file_name in tqdm(file_names_bw_3, 'onepiece grayscale'):
    shutil.copy(os.path.join(bw_3, file_name), target_dir_bw)
    os.rename(os.path.join(target_dir_bw, file_name), os.path.join(target_dir_bw, str('onepiece_bw') + file_name))

In [ ]:
print(len(os.listdir('./manga_dataset/colored/')))
print(len(os.listdir('./manga_dataset/grayscale/')))

In [ ]:
# colored
colored_files =  os.listdir('./manga_dataset/colored/')

# grayscale
grayscale_files =  os.listdir('./manga_dataset/grayscale/')
grayscale_files = grayscale_files[:3577]

In [ ]:
# shuffle the dataset
temp = list(zip(colored_files, grayscale_files))

random.shuffle(temp)
res1, res2 = zip(*temp)

# res1 and res2 come out as tuples, and so must be converted to lists.
colored, grayscale = list(res1), list(res2)

print(colored[0:5])
print(grayscale[0:5])

In [ ]:
#split the dataset 0.8 for training

#training colored
train_cl = []
for i in tqdm(range(int(0.0*len(colored)), int(0.8*len(colored)))):
    train_cl.append(colored[i])
    
#testing grayscale
test_cl = []
for i in tqdm(range(int(0.8*len(colored)), int(1.0*len(colored)))):
    test_cl.append(colored[i])

In [ ]:
print(train_cl[-3:None])
print(test_cl[0:3])

In [ ]:
#training grayscale
train_bw = []
for i in tqdm(range(int(0.0*len(grayscale)), int(0.8*len(grayscale)))):
    train_bw.append(grayscale[i])
    
#testing grayscale
test_bw = []
for i in tqdm(range(int(0.8*len(grayscale)), int(1.0*len(grayscale)))):
    test_bw.append(grayscale[i])

In [ ]:
print(train_bw[-3:None])
print(test_bw[0:3])

In [ ]:
# create the dataset folders
if not os.path.isdir('./train_test'):
    os.makedirs('./train_test')
    # unpaired image dataset path
    os.makedirs('./train_test/trainA/')
    os.makedirs('./train_test/trainB/')
    os.makedirs('./train_test/testA/')
    os.makedirs('./train_test/testB/')

In [ ]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [ ]:
# copy the train dataset to unpaired image dataset path
for i in tqdm(range(len(train_cl)), 'copying train colored and grayscale'):
    # Source path 
    src1 = find(str(train_bw[i]), './manga_dataset/grayscale/')
    src2 = find(str(train_cl[i]), './manga_dataset/colored/')

    # Destination path 
    dest1 = './train_test/trainA/'
    dest2 = './train_test/trainB/'

    # Copy the content of source to destination 
    shutil.copy(src1, dest1) 
    shutil.copy(src2, dest2)  

In [ ]:
# copy the test dataset to unpaired image dataset path
for i in tqdm(range(len(test_cl)), 'copying test colored and grayscale'):
    # Source path 
    src1 = find(str(test_bw[i]), './manga_dataset/grayscale/')
    src2 = find(str(test_cl[i]), './manga_dataset/colored/')

    # Destination path 
    dest1 = './train_test/testA/'
    dest2 = './train_test/testB/'

    # Copy the content of source to destination 
    shutil.copy(src1, dest1) 
    shutil.copy(src2, dest2) 

# Install MMGeneration

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

# Check Pytorch installation
import torch
print(torch.__version__)

In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!conda install pytorch==1.6.0 torchvision==0.7.0 cudatoolkit=10.1 -c pytorch -y

# install the latest mmcv
# pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.6.0/index.html

# install mmgeneration
!rm -rf mmgeneration
!git clone https://github.com/open-mmlab/mmgeneration.git
%cd mmgeneration
!pip install -r requirements.txt
!pip install -v -e .

In [ ]:
from mmcv import collect_env
collect_env()

In [ ]:
# Check MMDetection installation
import mmgen
print(mmgen.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Start training

In [ ]:
# download pre-trained cyclegan model in winter2summer dataset
import wget

url = 'https://download.openmmlab.com/mmgen/cyclegan/refactor/cyclegan_lsgan_resnet_in_1x1_246200_summer2winter_convert-bgr_20210902_165932-fcf08dc1.pth'
wget.download(url, out='../')

In [ ]:
%%writefile configs/cyclegan/cyclegan_lsgan_resnet_in_summer2winter_b1x1_250k.py

_base_ = [
    '../_base_/models/cyclegan/cyclegan_lsgan_resnet.py',
    '../_base_/datasets/unpaired_imgs_256x256.py',
    '../_base_/default_runtime.py'
]

domain_a = 'grayscale'
domain_b = 'colored'
model = dict(
    default_domain=domain_b,
    reachable_domains=[domain_a, domain_b],
    related_domains=[domain_a, domain_b],
    gen_auxiliary_loss=[
        dict(
            type='L1Loss',
            loss_weight=10.0,
            loss_name='cycle_loss',
            data_info=dict(
                pred=f'cycle_{domain_a}', target=f'real_{domain_a}'),
            reduction='mean'),
        dict(
            type='L1Loss',
            loss_weight=10.0,
            loss_name='cycle_loss',
            data_info=dict(
                pred=f'cycle_{domain_b}',
                target=f'real_{domain_b}',
            ),
            reduction='mean'),
        dict(
            type='L1Loss',
            loss_weight=0.5,
            loss_name='id_loss',
            data_info=dict(
                pred=f'identity_{domain_a}', target=f'real_{domain_a}'),
            reduction='mean'),
        dict(
            type='L1Loss',
            loss_weight=0.5,
            loss_name='id_loss',
            data_info=dict(
                pred=f'identity_{domain_b}', target=f'real_{domain_b}'),
            reduction='mean')
    ])
dataroot = '.././train_test/'
train_pipeline = [
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_a}',
        flag='color'),
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_b}',
        flag='color'),
    dict(
        type='Resize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        scale=(256, 368),
        interpolation='bicubic'),
    dict(
        type='Crop',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        crop_size=(368, 256),
        random_crop=True),
    dict(type='Flip', keys=[f'img_{domain_a}'], direction='horizontal'),
    dict(type='Flip', keys=[f'img_{domain_b}'], direction='horizontal'),
    dict(type='RescaleToZeroOne', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Normalize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        to_rgb=False,
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]),
    dict(type='ImageToTensor', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Collect',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        meta_keys=[f'img_{domain_a}_path', f'img_{domain_b}_path'])
]

test_pipeline = [
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_a}',
        flag='color'),
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key=f'img_{domain_b}',
        flag='color'),
    dict(
        type='Resize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        scale=(256, 368),
        interpolation='bicubic'),
    dict(type='RescaleToZeroOne', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Normalize',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        to_rgb=False,
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]),
    dict(type='ImageToTensor', keys=[f'img_{domain_a}', f'img_{domain_b}']),
    dict(
        type='Collect',
        keys=[f'img_{domain_a}', f'img_{domain_b}'],
        meta_keys=[f'img_{domain_a}_path', f'img_{domain_b}_path'])
]

data = dict(
    train=dict(
        dataroot=dataroot,
        pipeline=train_pipeline,
        domain_a=domain_a,
        domain_b=domain_b),
    val=dict(
        dataroot=dataroot,
        domain_a=domain_a,
        domain_b=domain_b,
        pipeline=test_pipeline),
    test=dict(
        dataroot=dataroot,
        domain_a=domain_a,
        domain_b=domain_b,
        pipeline=test_pipeline))

optimizer = dict(
    generators=dict(type='Adam', lr=0.0002, betas=(0.5, 0.999)),
    discriminators=dict(type='Adam', lr=0.0002, betas=(0.5, 0.999)))

# learning policy
lr_config = dict(
    policy='Linear', by_epoch=False, target_lr=0, start=125000, interval=1250)

checkpoint_config = dict(interval=5000, save_optimizer=True, by_epoch=False)
custom_hooks = [
    dict(
        type='MMGenVisualizationHook',
        output_dir='training_samples',
        res_name_list=[f'fake_{domain_a}', f'fake_{domain_b}'],
        interval=5000)
]

runner = None
use_ddp_wrapper = True
total_iters = 250000
load_from = '../cyclegan_lsgan_resnet_in_1x1_246200_summer2winter_convert-bgr_20210902_165932-fcf08dc1.pth'
# resume_from = './work_dirs/cyclegan_bw2cl/ckpt/cyclegan_bw2cl/iter_55000.pth'
workflow = [('train', 1)]
exp_name = 'cyclegan_bw2cl'
work_dir = f'./work_dirs/{exp_name}'
# testA: 716, testB:716
num_images = 716
metrics = dict(
    FID=dict(type='FID', num_images=num_images, image_shape=(3, 368, 256)),
    IS=dict(
        type='IS',
        num_images=num_images,
        image_shape=(3, 368, 256),
        inception_args=dict(type='pytorch')))

evaluation = dict(
    type='TranslationEvalHook',
    target_domain=domain_b,
    interval=5000,
    metrics=[
        dict(type='FID', num_images=num_images, bgr2rgb=True),
        dict(
            type='IS',
            num_images=num_images,
            inception_args=dict(type='pytorch'))
    ],
    best_metric=['fid', 'is'])

In [ ]:
!python tools/train.py configs/cyclegan/cyclegan_lsgan_resnet_in_summer2winter_b1x1_250k.py --work-dir ./work_dirs/cyclegan_bw2cl --no-validate